# 04. ARIMA Model

**Mô hình:** ARIMA(p, d, q)

**Thư viện:** statsmodels.tsa.arima.model.ARIMA

**Metrics:** RMSFE, MAPE

In [ ]:
# Import libraries

In [ ]:
# Load processed data

In [ ]:
# ARIMA model

In [ ]:
# Evaluation

In [ ]:
# Save results